# Web crawler - basic tools
<font size="1pt">This Jupyter notebook was created for the course Web Information Extraction and Retrieval at the University of Ljubljana. @szitnik</font>

## Fetching data from the Web

Python distribution already includes [*urllib* library](https://docs.python.org/3/library/urllib.html) which enables easy communication using HTTP requests. For those who would prefer a more feature-rich library we propose to take a look at the [*Requests* library](https://requests.readthedocs.io/en/master/). It is a high-level HTTP library and proposed to use by default for Python and supports multiple connections, sessions handling, proxies, etc. 

Let's retrieve [http://evem.gov.si](http://evem.gov.si) using a simple *urllib* library:

In [1]:
import urllib 


WEB_PAGE_ADDRESS = "http://evem.gov.si"

print(f"Retrieving web page URL '{WEB_PAGE_ADDRESS}'")

request = urllib.request.Request(
    WEB_PAGE_ADDRESS, 
    headers={'User-Agent': 'fri-ieps-TEST'}
)

with urllib.request.urlopen(request) as response: 
    html = response.read().decode("utf-8")
    print(f"Retrieved Web content: \n\n'\n{html}\n'")
    

Retrieving web page URL 'http://evem.gov.si'


URLError: <urlopen error [Errno 11001] getaddrinfo failed>

We can observe that the Web content we recieve is not expected. The HTML code includes a Javascript (JS) code that Web browser normally executes. The JS code above would *redirect* browser to the [*http://evem.gov.si/evem/drzavljani/zacetna.evem*](http://evem.gov.si/evem/drzavljani/zacetna.evem).

Libraries that enable us such functionality should automatically execute Javascript code. Generally, for this purpose they simulate browser such as Google Chrome or Firefox. An example of such library is [Selenium](https://www.selenium.dev/) ([Python API](https://selenium-python.readthedocs.io/)).

Selenium supports multiple browser drivers, so let's download and use a [ChromeDriver](https://sites.google.com/a/chromium.org/chromedriver/). After that we can try to visit the eVem Web page again but now using Selenium.

In [14]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

WEB_DRIVER_LOCATION = "C:/Users/Pirk/Desktop/faks-mag/ekstrakcija/chromedriver.exe"
TIMEOUT = 5

chrome_options = Options()
# If you comment the following line, a browser will show ...
#chrome_options.add_argument("--headless")

#Adding a specific user agent
chrome_options.add_argument("user-agent=fri-ieps-TEST")

print(f"Retrieving web page URL '{WEB_PAGE_ADDRESS}'")
driver = webdriver.Chrome(WEB_DRIVER_LOCATION, options=chrome_options)
driver.get(WEB_PAGE_ADDRESS)

# Timeout needed for Web page to render (read more about it)
time.sleep(TIMEOUT)

html = driver.page_source

print(f"Retrieved Web content (truncated to first 900 chars): \n\n'\n{html[:900]}\n'\n")

page_msg = driver.find_element_by_class_name("inside-text")

print(f"Web page message: '{page_msg.text}'")

driver.close()

Retrieving web page URL 'http://evem.gov.si'
Retrieved Web content (truncated to first 900 chars): 

'
<html lang="sl" class=" js rgba"><head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <meta content="IE=11.0000" http-equiv="X-UA-Compatible">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    
    <base href="http://evem.gov.si/evem/">
    <title>
      SPOT - Državni portal za poslovne subjekte in samostojne podjetnike
    </title>
    <script language="JavaScript" type="text/JavaScript">
  var _staticResourcesRoot = 'http://evem.gov.si/evem_static/';
</script>
    <link rel="shortcut icon" href="http://evem.gov.si/evem_static/favicon.ico?v=3">
    <link rel="apple-touch-icon" type="image/png" sizes="180x180" href="http://evem.gov.si/evem_static/apple-touch-icon.png">
	<link rel="icon" type="image/png" sizes="32x32" href="http://evem.gov.si/evem_static/favicon-32x32.png">
	<link rel="icon" type="image/png" si

## PostgreSQL database

### System initialization

In this tutorial we use PostgreSQL database Docker image (Windows users should use docker's IP instead of *localhost* below). You can also install the Database server locally to your machine.

First we prepare a file *database.sql*. The script will create a table with two rows:

```
CREATE SCHEMA IF NOT EXISTS showcase;

CREATE TABLE showcase.counters (
    counter_id integer  NOT NULL,
    value integer NOT NULL,
    CONSTRAINT pk_counters PRIMARY KEY ( counter_id )
 );

INSERT INTO showcase.counters VALUES (1,0), (2,0);
```

Go to an empty folder and save the script into a subfolder named *init_scripts*. Create another empty folder named *pgdata*.

We run docker container using the following command. The command will name the container *postgresql-wier*, set username and password, map database files to folder *./pgdata* and initialization scripts to *./init-scripts*, map port 5432 to host machine (i.e. localhost) and run image *postgres:9* in a detached mode. 

```
docker run --name postgresql-wier \
    -e POSTGRES_PASSWORD=SecretPassword \
    -e POSTGRES_USER=user \
    -v $PWD/pgdata:/var/lib/postgresql/data \
    -v $PWD/init-scripts:/docker-entrypoint-initdb.d \
    -p 5432:5432 \
    -d postgres:12.2
```

To check container's logs, run `docker logs -f postgresql-wier`.

To log into the database and execute SQL statements, run the following command: `docker exec -it postgresql-wier psql -U user`.

### Database example

Idea of the example below is to show how to implement concurrency and connect to a PostgreSQL database. 

We create three workers that run function *increase_db_values* and three workers that run function *increase_db_values_locking*. The first three workers will execute accesses to database interleavingly while the second three workers will enter into the database value update block in order.

In [15]:
import concurrent.futures
import threading
import psycopg2

lock = threading.Lock()

def reset_db_values():
    conn = psycopg2.connect(host="http://83.212.127.54/", user="test", password="fricrawl")
    conn.autocommit = True
    
    cur = conn.cursor()
    cur.execute("UPDATE showcase.counters SET value = 0")
    
    cur.close()
    conn.close()
    
def print_db_values():
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True

    print("\nValues in the database:")
    cur = conn.cursor()
    cur.execute("SELECT counter_id, value FROM showcase.counters ORDER BY counter_id")
    for counter_id, value in cur.fetchall():
        print(f"\tCounter id: {counter_id}, value: {value}")
    cur.close()
    conn.close()

def increase_db_values(counter_id, increases):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True
    
    for i in range(increases):
        cur = conn.cursor()
        cur.execute("SELECT value FROM showcase.counters WHERE counter_id = %s", \
                    (counter_id,))
        value = cur.fetchone()[0]
        cur.execute("UPDATE showcase.counters SET value = %s WHERE counter_id = %s", \
                    (value+1, counter_id))
        cur.close()
    conn.close()
    
def increase_db_values_locking(counter_id, increases):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True
    
    for i in range(increases):
        with lock:
            cur = conn.cursor()
            cur.execute("SELECT value FROM showcase.counters WHERE counter_id = %s", \
                        (counter_id,))
            value = cur.fetchone()[0]
            cur.execute("UPDATE showcase.counters SET value = %s WHERE counter_id = %s", \
                        (value+1, counter_id))
            cur.close()
    conn.close()

reset_db_values()
print_db_values()

with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
    print(f"\n ... executing workers ...\n")
    for _ in range(3):
        executor.submit(increase_db_values, 1,1000)
    for _ in range(3):
        executor.submit(increase_db_values_locking, 2,1000)
    
print_db_values()

OperationalError: could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 5432?
could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 5432?
